### Parse data from text-based pdf file
1. pdf to docx: pdf2docx
2. docx to json: handcraft python program

Author: Roger

#### .pdf to .docx

In [ ]:
from pdf2docx import Converter

file_name = '嘉市個案評估+照顧計畫1'

pdf_file = f'./data/{file_name}.pdf'
docx_file = f'./results/pdf2docx_{file_name}.docx'

# convert pdf to docx
cv = Converter(pdf_file)
cv.convert(docx_file)
cv.close()

#### table in .docx to json

In [ ]:
from docx import Document, table
import json

def extract_cell_value(celldata):
    if celldata.tables:
        cell_data = []
        for table in celldata.tables:
            for row_idx, row in enumerate(table.rows):
                for col_idx, cell in enumerate(row.cells):
                    cell_text = cell.text.strip()
                    cell_data.append(cell_text)
        return cell_data
    else:
        return [celldata.text.replace('\n', '').strip()]

def doc_tables_to_dict(doc_path):
    doc = Document(doc_path)
    all_data = []
    
    # TODO:
    # check image in doc and do OCR
    
    # parse data from tables
    for table in doc.tables:
        table_data = {}
        for row in table.rows:
            if len(row.cells) % 2 == 1:
                print('Column counts WARNING')
                pass
            else:
                for i in range(0, len(row.cells), 2):
                    key = extract_cell_value(row.cells[i])
                    value = extract_cell_value(row.cells[i+1])
                    
                    if len(key) != 1:
                        if key == value:
                            continue
                    key = key[0]
                    
                    table_data[key] = value
                    print(key, '\n', value, '\n')
        all_data.append(table_data)

    print('Paragraphs:')
    for paragraph in doc.paragraphs:
        for run in paragraph.runs:
            print(run.text)
    
    return all_data

def save_dict_to_json(data, json_path):
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

doc_path = f'results/pdf2docx_{file_name}.docx'
json_path = f'results/pdf2docx_{file_name}.json'
data = doc_tables_to_dict(doc_path)
save_dict_to_json(data, json_path)

print("Finish parsing key-value pair")